# Auditoría de Integridad y Métricas — DataSecure Lab

Este notebook presenta las métricas reales generadas durante la ejecución del pipeline de integridad de datos en HDFS.

**Fecha de ejecución:** 2026-02-13  
**Clúster:** 3 DataNodes

## 1) Configuración y carga de datos

In [1]:
from pathlib import Path
import pandas as pd
import re

# Rutas a los directorios de auditoría
AUDIT_DIR = Path('/media/notebooks/audit/fsck')
INVENTORY_DIR = Path('/media/notebooks/audit/inventory')
METRICS_DIR = Path('/media/notebooks/audit/metrics')

print('=' * 50)
print('VERIFICACIÓN DE DIRECTORIOS')
print('=' * 50)
print(f'AUDIT_DIR exists:     {AUDIT_DIR.exists()}')
print(f'INVENTORY_DIR exists: {INVENTORY_DIR.exists()}')
print(f'METRICS_DIR exists:   {METRICS_DIR.exists()}')

# Listar auditorías disponibles
if AUDIT_DIR.exists():
    print(f'\nAuditorías disponibles:')
    for f in sorted(AUDIT_DIR.rglob('*.txt')):
        print(f'  - {f.relative_to(AUDIT_DIR)}')

VERIFICACIÓN DE DIRECTORIOS
AUDIT_DIR exists:     True
INVENTORY_DIR exists: True
METRICS_DIR exists:   True

Auditorías disponibles:
  - 2026-02-13/fsck_data.txt
  - 2026-02-13/fsck_post_incident.txt
  - 2026-02-13/fsck_pre_incident.txt


## 2) Resumen de Auditoría FSCK

Análisis del estado de integridad del sistema de ficheros HDFS.

In [2]:
def parse_fsck(text):
    """Extrae métricas de una salida de hdfs fsck."""
    def extract(pattern, default='0'):
        match = re.search(pattern, text)
        return match.group(1) if match else default
    
    return {
        'Total size (bytes)': extract(r'Total size:\s+(\d+)'),
        'Total files': extract(r'Total files:\s+(\d+)'),
        'Total blocks': extract(r'Total blocks[^:]*:\s+(\d+)'),
        'DataNodes': extract(r'Number of data-nodes:\s+(\d+)'),
        'Corrupt blocks': extract(r'Corrupt blocks:\s+(\d+)'),
        'Missing blocks': extract(r'Missing blocks:\s+(\d+)'),
        'Under-replicated': extract(r'Under-replicated blocks:\s+(\d+)'),
        'Replication factor': extract(r'Default replication factor:\s+(\d+)'),
        'Status': 'HEALTHY' if 'HEALTHY' in text else 'CORRUPT' if 'CORRUPT' in text.upper() else 'UNKNOWN'
    }

# Cargar auditoría principal
fsck_file = AUDIT_DIR / '2026-02-13' / 'fsck_data.txt'
if fsck_file.exists():
    fsck_text = fsck_file.read_text()
    fsck_metrics = parse_fsck(fsck_text)
    
    print('=' * 50)
    print('RESUMEN DE AUDITORÍA FSCK - /data')
    print('=' * 50)
    for k, v in fsck_metrics.items():
        print(f'{k:25} : {v}')
else:
    print('No se encontró fsck_data.txt')

RESUMEN DE AUDITORÍA FSCK - /data
Total size (bytes)        : 816636433
Total files               : 2
Total blocks              : 13
DataNodes                 : 3
Corrupt blocks            : 0
Missing blocks            : 0
Under-replicated          : 0
Replication factor        : 3
Status                    : HEALTHY


## 3) Métricas de Replicación

Comparación del impacto de diferentes factores de replicación (1, 2, 3) sobre el almacenamiento y tiempos.

In [3]:
# Cargar métricas de replicación desde CSV
rep_csv = METRICS_DIR / '2026-02-13' / 'replication_metrics.csv'

if rep_csv.exists():
    df_rep = pd.read_csv(rep_csv)
    
    # Añadir columnas calculadas
    df_rep['Espacio lógico (MB)'] = (df_rep['logical_size_bytes'] / 1024 / 1024).round(2)
    df_rep['Espacio físico (MB)'] = df_rep['physical_size_mb']
    df_rep['Multiplicador'] = (df_rep['physical_size_bytes'] / df_rep['logical_size_bytes']).round(1)
    df_rep['Tolerancia a fallos'] = df_rep['factor'].map({
        1: 'Sin tolerancia',
        2: 'Tolera 1 fallo',
        3: 'Tolera 1 fallo + re-replicación segura'
    })
    
    # Mostrar tabla formateada
    print('=' * 90)
    print('IMPACTO DE REPLICACIÓN EN ALMACENAMIENTO')
    print('=' * 90)
    display_df = df_rep[['factor', 'Espacio lógico (MB)', 'Espacio físico (MB)', 
                         'Multiplicador', 'time_setrep_sec', 'Tolerancia a fallos']].copy()
    display_df.columns = ['Factor', 'Lógico (MB)', 'Físico (MB)', 'Multiplicador', 'Tiempo (s)', 'Tolerancia']
    print(display_df.to_string(index=False))
    
    # Mostrar gráfico de barras ASCII
    print('\n' + '=' * 50)
    print('GRÁFICO: Espacio físico vs Factor de replicación')
    print('=' * 50)
    max_val = df_rep['physical_size_mb'].max()
    for _, row in df_rep.iterrows():
        bar_len = int(40 * row['physical_size_mb'] / max_val)
        bar = '█' * bar_len
        print(f"Factor {int(row['factor'])}: {bar} {int(row['physical_size_mb'])} MB")
else:
    print('No se encontró replication_metrics.csv')

IMPACTO DE REPLICACIÓN EN ALMACENAMIENTO
 Factor  Lógico (MB)  Físico (MB)  Multiplicador  Tiempo (s)                             Tolerancia
      1       376.33          376            1.0          41                         Sin tolerancia
      2       376.33          752            2.0          11                         Tolera 1 fallo
      3       376.33         1128            3.0          11 Tolera 1 fallo + re-replicación segura

GRÁFICO: Espacio físico vs Factor de replicación
Factor 1: █████████████ 376 MB
Factor 2: ██████████████████████████ 752 MB
Factor 3: ████████████████████████████████████████ 1128 MB


## 4) Uso de Recursos (Docker Stats)

In [4]:
# Cargar docker stats
stats_file = METRICS_DIR / '2026-02-13' / 'docker_stats.txt'

if stats_file.exists():
    stats_text = stats_file.read_text()
    print('=' * 95)
    print('USO DE RECURSOS - DOCKER STATS')
    print('=' * 95)
    print(stats_text)
else:
    print('No se encontró docker_stats.txt')

USO DE RECURSOS - DOCKER STATS
NAME                           CPU %     MEM USAGE / LIMIT     NET I/O           BLOCK I/O
clustera-dnnm-3                0.32%     819.5MiB / 15.15GiB   1.65GB / 1.51GB   74.2MB / 1.65GB
clustera-dnnm-1                0.40%     804.4MiB / 15.15GiB   1.65GB / 1.38GB   15.7MB / 1.65GB
clustera-dnnm-2                0.46%     795.8MiB / 15.15GiB   1.65GB / 1.24GB   105MB / 1.65GB
namenode                       0.15%     655.7MiB / 15.15GiB   830MB / 1.66GB    226MB / 4.42MB
resourcemanager                0.42%     551.4MiB / 15.15GiB   1.94MB / 567kB    99.9MB / 1.68MB
DB-ja-changes-andalucia-2024   0.00%     35.74MiB / 15.15GiB   1.63kB / 126B     55.2MB / 4.1kB



## 5) Validación de Backup

In [5]:
# Cargar informe de inventario
inv_file = INVENTORY_DIR / '2026-02-13' / 'inventory_report_2026-02-13.txt'

if inv_file.exists():
    inv_text = inv_file.read_text()
    print('=' * 50)
    print('VALIDACIÓN DE BACKUP')
    print('=' * 50)
    print(inv_text)
else:
    print('No se encontró inventory_report.txt')

VALIDACIÓN DE BACKUP
========== INFORME DE COMPARACIÓN ==========
Fecha: 2026-02-13

Ficheros en origen:  2
Ficheros en backup:  2


========== RESUMEN ==========
Coincidentes: 2
Missing:      0
Mismatch:     0

RESULTADO: BACKUP CONSISTENTE



## 6) Análisis del Incidente

Comparación del estado ANTES y DESPUÉS de la caída simulada de un DataNode.

In [6]:
# Comparar auditorías pre y post incidente
pre_file = AUDIT_DIR / '2026-02-13' / 'fsck_pre_incident.txt'
post_file = AUDIT_DIR / '2026-02-13' / 'fsck_post_incident.txt'

print('=' * 70)
print('ANÁLISIS DEL INCIDENTE - CAÍDA DE DATANODE')
print('=' * 70)

if pre_file.exists() and post_file.exists():
    pre_metrics = parse_fsck(pre_file.read_text())
    post_metrics = parse_fsck(post_file.read_text())
    
    # Crear tabla comparativa
    comparison = []
    for key in ['DataNodes', 'Status', 'Corrupt blocks', 'Missing blocks', 'Under-replicated']:
        comparison.append({
            'Métrica': key,
            'ANTES': pre_metrics.get(key, 'N/A'),
            'DESPUÉS': post_metrics.get(key, 'N/A')
        })
    
    df_incident = pd.DataFrame(comparison)
    print(df_incident.to_string(index=False))
    
    print('\nNota: HDFS tarda ~15 minutos en detectar un DataNode como "dead".')
    print('Durante ese periodo, los bloques aún aparecen como disponibles.')
else:
    print('Archivos de incidente no encontrados.')
    print('Ejecuta: bash scripts/70_incident_simulation.sh')

ANÁLISIS DEL INCIDENTE - CAÍDA DE DATANODE
         Métrica   ANTES DESPUÉS
       DataNodes       3       3
          Status HEALTHY HEALTHY
  Corrupt blocks       0       0
  Missing blocks       0       0
Under-replicated       0       0

Nota: HDFS tarda ~10 minutos en detectar un DataNode como "dead".
Durante ese periodo, los bloques aún aparecen como disponibles.


## 7) Conclusiones y Recomendaciones

In [7]:
print('=' * 70)
print('CONCLUSIONES Y RECOMENDACIONES')
print('=' * 70)

conclusions = '''
FACTOR DE REPLICACIÓN RECOMENDADO: 3
   - Con 3 DataNodes, replicación 3 garantiza que cada bloque esté en todos los nodos
   - Coste: 3x espacio en disco (1128 MB físicos por 376 MB lógicos)
   - Beneficio: Tolerancia a la caída de 1 nodo sin pérdida de datos
   - La re-replicación automática protege ante fallos durante recuperación

FRECUENCIA DE AUDITORÍA RECOMENDADA: Diaria
   - `hdfs fsck` es una operación de solo lectura (no impacta rendimiento)
   - Permite detectar bloques UNDER_REPLICATED o CORRUPT tempranamente
   - En producción con mayor volumen: semanal + alertas automáticas del NameNode

ESTRATEGIA DE BACKUP:
   - Backup validado: 2 ficheros coincidentes, 0 missing, 0 mismatch
   - Recomendación: Backup diario a /backup con validación por inventario
   - Considerar backup a segundo clúster (DistCp) para DR real

LECCIONES DEL INCIDENTE:
   - HDFS detecta caída de DataNode en ~10 minutos (configurable)
   - Con replicación 3, los datos permanecen accesibles durante el incidente
   - La recuperación es automática al reiniciar el DataNode
'''
print(conclusions)

CONCLUSIONES Y RECOMENDACIONES

FACTOR DE REPLICACIÓN RECOMENDADO: 3
   - Con 3 DataNodes, replicación 3 garantiza que cada bloque esté en todos los nodos
   - Coste: 3x espacio en disco (1128 MB físicos por 376 MB lógicos)
   - Beneficio: Tolerancia a la caída de 1 nodo sin pérdida de datos
   - La re-replicación automática protege ante fallos durante recuperación

FRECUENCIA DE AUDITORÍA RECOMENDADA: Diaria
   - `hdfs fsck` es una operación de solo lectura (no impacta rendimiento)
   - Permite detectar bloques UNDER_REPLICATED o CORRUPT tempranamente
   - En producción con mayor volumen: semanal + alertas automáticas del NameNode

ESTRATEGIA DE BACKUP:
   - Backup validado: 2 ficheros coincidentes, 0 missing, 0 mismatch
   - Recomendación: Backup diario a /backup con validación por inventario
   - Considerar backup a segundo clúster (DistCp) para DR real

LECCIONES DEL INCIDENTE:
   - HDFS detecta caída de DataNode en ~10 minutos (configurable)
   - Con replicación 3, los datos perma

## 8) Tabla Resumen de Métricas

In [8]:
# Tabla resumen final con datos reales
summary_data = {
    'Métrica': [
        'Tamaño total datos (logs + IoT)',
        'Ficheros en /data',
        'Bloques totales',
        'Factor de replicación',
        'DataNodes activos',
        'Bloques corruptos',
        'Bloques missing',
        'Bloques under-replicated',
        'Estado del filesystem',
        'Backup validado'
    ],
    'Valor': [
        '~779 MB (816,636,433 bytes)',
        '2 (logs_20260213.log + iot_20260213.jsonl)',
        '13',
        '3',
        '3',
        '0',
        '0',
        '0',
        'HEALTHY',
        'Sí (2 ficheros, 0 errores)'
    ]
}

df_summary = pd.DataFrame(summary_data)
print('=' * 60)
print('RESUMEN EJECUTIVO')
print('=' * 60)
print(df_summary.to_string(index=False))

RESUMEN EJECUTIVO
                        Métrica                                      Valor
Tamaño total datos (logs + IoT)                ~779 MB (816,636,433 bytes)
              Ficheros en /data 2 (logs_20260213.log + iot_20260213.jsonl)
                Bloques totales                                         13
          Factor de replicación                                          3
              DataNodes activos                                          3
              Bloques corruptos                                          0
                Bloques missing                                          0
       Bloques under-replicated                                          0
          Estado del filesystem                                    HEALTHY
                Backup validado                 Sí (2 ficheros, 0 errores)


In [9]:
# Exportar tablas a CSV
print('\nExportando tablas a CSV...')
df_summary.to_csv('/media/notebooks/resumen_ejecutivo.csv', index=False)
print('Exportado: /media/notebooks/resumen_ejecutivo.csv')

if 'df_rep' in dir():
    df_rep.to_csv('/media/notebooks/replicacion_comparativa.csv', index=False)
    print('Exportado: /media/notebooks/replicacion_comparativa.csv')


Exportando tablas a CSV...
Exportado: /media/notebooks/resumen_ejecutivo.csv
Exportado: /media/notebooks/replicacion_comparativa.csv
